<a href="https://colab.research.google.com/github/JRCon1/Google-Sheets-Automated-Portfolio-Tracker-/blob/main/PortfolioTrackerScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
#Install all required packages. Use %%capture to hide download output and warnings

%%capture
!pip install empyrial
!pip install quantstats
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
from google.colab import auth
import gspread
from google.auth import default
import empyrial as ep
from empyrial import *
import quantstats as qs
qs.extend_pandas()

In [15]:
#Google Sheet Linkage Portion

# Authenticate to Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

#Link the Spreadsheet URL, the Spreadsheet, and Specify which sheet you're refering too (starting from 0)
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1loKt4k5q7tXFvrlgLloPcnvnuCCromIjADUvxqxyksU/edit?gid=0#gid=0"
spreadsheet = gc.open_by_url(spreadsheet_url)
#Select the second sheet
sheet = spreadsheet.get_worksheet(1)

In [16]:
%%capture
#Store the tickers and the weights from the sheet into lists in preparation for the backtest
tickers = [ticker[0] for ticker in sheet.get('A2:A15')]
weights = [float(weight[0].strip('%')) / 100 for weight in sheet.get('L2:L15')]

#Set a start date to run the backtest from, and perform it.
start = dt.date(2019, 8, 14)
p = Engine(start_date= start, portfolio = tickers, weights = weights, benchmark = ['SPY'])
empyrial(p)

display_sheet = spreadsheet.get_worksheet(0)
data = empyrial.df
quant_stats = pd.DataFrame(data)
QSData = [quant_stats.columns.tolist()] + quant_stats.values.tolist()
display_sheet.update('C7', QSData)

In [18]:
# Download Data and Calculate Returns
df = yf.download(tickers, start=start)['Adj Close']
df = df.reindex(columns=tickers)
returns = df.pct_change().dropna()

# Define Risk Free Rate
rf = 0.03

# Function to calculate, round, convert to list, and update sheet
def calculate_and_update(stat_function, column, *args, decimals=2, **kwargs):
    stat_values = stat_function(returns, *args, **kwargs)
    rounded_values = np.round(stat_values, decimals).tolist()
    for i, value in enumerate(rounded_values):
        cell = f'{column}{2 + i}'
        sheet.update_acell(cell, value)

# Update CAGR
calculate_and_update(qs.stats.cagr, 'Q')

# Update Volatility
calculate_and_update(qs.stats.volatility, 'R')

# Update Sharpe Ratio
calculate_and_update(qs.stats.sharpe, 'S', rf=rf)

# Update Sortino Ratio
calculate_and_update(sortino_ratio, 'T', period='daily')

# Update Calmar Ratio
calculate_and_update(qs.stats.calmar, 'U', decimals=2)

# Update VaR
calculate_and_update(qs.stats.var, 'V', sigma=1, confidence=0.95, decimals=4)

# Update CVaR
calculate_and_update(qs.stats.cvar, 'W', sigma=1, confidence=0.95, decimals=4)

# Update Max Drawdown
calculate_and_update(max_drawdown, 'X', out=None, decimals=2)

#Update 30 Day Rolling Implied Volatility
iv = qs.stats.implied_volatility(returns).rolling(30).mean().iloc[-1:]
iv_percentages = np.round(iv.values[0] * 100, 2).tolist()
sheet.update_acell('Y2', f"{iv_percentage}%")

[*********************100%%**********************]  14 of 14 completed


In [19]:
#Add or Update Yearly Dividend Income by Stock
dividends = pd.DataFrame()
for ticker in tickers:
    dividends[ticker] = yf.Ticker(ticker).dividends.iloc[-4:].reset_index(drop=True)
    dividends[pd.isna(dividends)] = 0
dividends = dividends.sum().to_list()

for i, value in enumerate(dividends):
    cell = f'M{2 + i}'
    sheet.update_acell(cell, value)

In [20]:
#Create and then store Dividend Calendar
shares = [float(share[0]) for share in sheet.get('C2:C15')]

months = pd.date_range(start='2023-08-01', end='2024-07-31', freq='M').strftime('%Y-%m').tolist()
dividends = pd.DataFrame(columns=months)

for i, ticker in enumerate(tickers):
    ticker_data = yf.Ticker(ticker).dividends
    monthly_dividends = ticker_data.groupby(ticker_data.index.to_period('M')).sum()
    monthly_dividends.index = monthly_dividends.index.strftime('%Y-%m')
    dividends.loc[ticker] = monthly_dividends.reindex(months, fill_value=0).values * shares[i]

month_names = pd.date_range(start='2023-08-01', end='2024-07-31', freq='M').strftime('%B').tolist()
dividends.columns = month_names

output = [['Dividend Calendar'] + month_names]
output += [[ticker] + dividends.loc[ticker].tolist() for ticker in tickers]

sheet3 = spreadsheet.get_worksheet(2)
sheet3.update('B2', output)

{'spreadsheetId': '1loKt4k5q7tXFvrlgLloPcnvnuCCromIjADUvxqxyksU',
 'updatedRange': "'Dividend Calender'!B2:N16",
 'updatedRows': 15,
 'updatedColumns': 13,
 'updatedCells': 195}

In [21]:
#Add in Sectors
sectors = []
for ticker in tickers:
    sector = yf.Ticker(ticker).info.get('sector', 'N/A')
    if sector == 'N/A':
        sector = 'ETF'
    sectors.append(sector)

for i, sector in enumerate(sectors):
    cell = f'B{2 + i}'
    sheet.update_acell(cell, sector)

In [22]:
#Prepare Plotting
portfolio_returns = (returns * weights).sum(axis=1)


# Calculate the cumulative return for the portfolio
portfolio_cum_returns = (1 + portfolio_returns).cumprod()
benchmark_cum_returns = (1 + returns['VOO']).cumprod()

cum_returns_df = pd.DataFrame({
    'Date': portfolio_cum_returns.index.strftime('%Y-%m-%d'),
    'Portfolio': portfolio_cum_returns.values,
    'Benchmark': benchmark_cum_returns.values
})

# Convert the DataFrame to a list of lists for updating Google Sheets
output_data = [cum_returns_df.columns.tolist()] + cum_returns_df.values.tolist()

# Update the Google Sheet starting from B150
sheet3.update('B150', output_data)

#Individual Returns Portion
cumulative_returns  = (1 + returns).cumprod()
net_percent_returns = (cumulative_returns - 1)
net_percent_returns.index = net_percent_returns.index.strftime('%Y-%m-%d')
output_dataInd = [net_percent_returns.columns.tolist()] + net_percent_returns.values.tolist()
sheet3.update('K150', output_dataInd)

{'spreadsheetId': '1loKt4k5q7tXFvrlgLloPcnvnuCCromIjADUvxqxyksU',
 'updatedRange': "'Dividend Calender'!K150:X1411",
 'updatedRows': 1262,
 'updatedColumns': 14,
 'updatedCells': 17668}